In [1]:
import os
import dotenv
import pandas as pd
import geopandas as gpd
import pyeuropeana.apis as apis
import pyeuropeana.utils as utils

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from utils import get_provider_data, aggregate_location_counts

In [3]:
dotenv.load_dotenv()

True

In [4]:
print(os.getenv('EUROPEANA_API_KEY'))

snetrourst


In [5]:
# Load world map dataset
world = gpd.read_file("lowres_world.geojson")

In [6]:
df = get_provider_data("Museum of World Culture", world)

/anaconda/envs/glamhack24/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
world

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,889953.0,Oceania,Fiji,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,58005463.0,Africa,Tanzania,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253.0,Africa,W. Sahara,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,37589262.0,North America,Canada,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,328239523.0,North America,United States of America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
...,...,...,...,...,...,...
172,6944975.0,Europe,Serbia,SRB,51475,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
173,622137.0,Europe,Montenegro,MNE,5542,"POLYGON ((20.07070 42.58863, 19.80161 42.50009..."
174,1794248.0,Europe,Kosovo,-99,7926,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."
175,1394973.0,North America,Trinidad and Tobago,TTO,24269,"POLYGON ((-61.68000 10.76000, -61.10500 10.890..."


In [8]:
location_counts = aggregate_location_counts(df)
print(location_counts)

   object_location  count
0           Sweden    944
1          Bolivia     29
2          Germany      6
3         Colombia      4
4           France      3
5           Panama      3
6             Peru      3
7      Solomon Is.      2
8           Mexico      2
9        Argentina      1
10          Brazil      1
11           Chile      1
12         Unknown      1
